In [ ]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv

import random
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split

In [ ]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        yield l

In [ ]:
interactions = open("trainInteractions.tsv", 'w')
for l in readCSV("trainInteractions.csv.gz"):
    user,recipeId,date,rating = l
    l1 = "\t".join([user,recipeId,rating])
    interactions.write(l1+"\n")

In [ ]:
from surprise import SVD, NormalPredictor
from surprise.model_selection import GridSearchCV

reader = Reader(line_format='user item rating', sep='\t')
data = Dataset.load_from_file("trainInteractions.tsv", reader=reader)

param_grid = {'reg_all':[0.5,0.01,0.1,.05,.005,.001]}
gs = GridSearchCV(SVD, param_grid, measures=['mse'], cv=3)
gs.fit(data)
params = gs.best_params['mse']
svdtuned = SVD(n_factors=params['n_factors'], n_epochs=params['n_epochs'],lr_all=params['lr_all'], reg_all=params['reg_all'])

In [ ]:
print(params)

In [ ]:
svdtuned.fit(data.build_full_trainset())

In [ ]:
# reader = Reader(line_format='user item rating', sep='\t')
# data = Dataset.load_from_file("trainInteractions.tsv", reader=reader)
# model = SVD()
# model.fit(data.build_full_trainset())

In [ ]:
# allRatings = []
# userRatings = defaultdict(list)

# for user,recipe,d in readCSV("trainInteractions.csv.gz"):
#   r = int(d['rating'])
#   allRatings.append(r)
#   userRatings[user].append(r)

# globalAverage = sum(allRatings) / len(allRatings)
# userAverage = {}
# for u in userRatings:
#   userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

predictionFile = open("predictions_Rated.txt", 'w')
for l in open("stub_Rated.txt"):
    if l.startswith("user_id"):
        #header
        predictionFile.write(l)
        continue
    u,i = l.strip().split('-')
    pred = svdtuned.predict(u,i)
    predictionFile.write(u + '-' + i + ',' + str(pred.est) + '\n')

predictionFile.close()

